In [1]:

import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


残差块

In [3]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super(Residual, self).__init__()
        
        self.main_path = nn.Sequential(
            nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides),
            nn.BatchNorm2d(num_channels),
            nn.ReLU(),
            nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_channels)
        )
        
        self.shortcut_path = nn.Sequential()
        if use_1x1conv:
            self.shortcut_path = nn.Sequential(
                nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
            )
    
    def forward(self, input):
        output = self.main_path(input)
        shortcut = self.shortcut_path(input)
        output += shortcut
        return F.relu(output)

In [4]:
os.makedirs(r"/home/cxmd/文档/git_files/neural_network/ResNet/run1", exist_ok=True)
residual = Residual(input_channels=3, num_channels=3, use_1x1conv=True)
print(residual)
writer = SummaryWriter('/home/cxmd/文档/git_files/neural_network/ResNet/run1/Residual')
writer.add_graph(residual, torch.randn(1, 3, 32, 32))
writer.close()

Residual(
  (main_path): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut_path): Sequential(
    (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  )
)


In [5]:
X = torch.rand(4, 3, 6, 6)
Y = residual(X)
Y.shape

torch.Size([4, 3, 6, 6])

ResNet模型

In [6]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), 
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
print(b1)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)


In [7]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [8]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [9]:
resnet = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [10]:
writer = SummaryWriter('/home/cxmd/文档/git_files/neural_network/ResNet/run1/ResNet')
writer.add_graph(resnet, torch.randn(64, 1, 7, 7))
writer.close()

In [11]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in resnet:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
